What is Concurrency?

Concurrency is when multiple tasks are in progress at the same time. Python provides:
•	Threading → Multiple threads in a single process (shared memory)
•	Multiprocessing → Multiple processes (separate memory, true parallelism)


What is Threading?
Threading allows your program to run multiple operations concurrently within the same process.

    Useful for I/O-bound tasks (e.g., file logging, API calls, database writes).

    Not ideal for CPU-bound tasks (due to the Global Interpreter Lock – GIL in CPython).

In [1]:
import threading
import time

def download_file(file):
    print(f"Started downloading {file}")
    time.sleep(2)
    print(f"Finished downloading {file}")

# Creating threads
t1 = threading.Thread(target=download_file, args=("file1.txt",))
t2 = threading.Thread(target=download_file, args=("file2.txt",))

# Start threads
t1.start()
t2.start()

# Wait for completion
t1.join()
t2.join()


print("Both downloads completed!")

Started downloading file1.txtStarted downloading file2.txt

Finished downloading file2.txt
Finished downloading file1.txt
Both downloads completed!


Threads share the same memory space and run concurrently (not in true parallel on CPU due to the GIL).

Bank Application Use Case

Imagine a banking system where:
1.	You process transactions
2.	You log each transaction
3.	You audit sensitive activities

We’ll simulate concurrent logging and auditing using threads.

In [2]:
import threading
import time

class BankAccount:
    def __init__(self, name, balance=0):
        self.name = name
        self.balance = balance
        self.lock = threading.Lock()  # For thread-safe access

    def deposit(self, amount):
        with self.lock:
            self.balance += amount
            print(f"[{threading.current_thread().name}] Deposited ₹{amount}, New Balance: ₹{self.balance}")

    def withdraw(self, amount):
        with self.lock:
            if amount > self.balance:
                print(f"[{threading.current_thread().name}] Insufficient funds.")
            else:
                self.balance -= amount
                print(f"[{threading.current_thread().name}] Withdrew ₹{amount}, New Balance: ₹{self.balance}")


In [5]:
#Logging Function (Runs in a separate thread)

def log_transaction(action, amount):
    time.sleep(0.5)  # Simulate delay
    print(f"[LOG] {action} of ₹{amount} logged.")

In [6]:
#Auditing Function (Runs in a separate thread)

def audit(action, amount):
    time.sleep(1)  # Simulate delay
    print(f"[AUDIT] {action.upper()} of ₹{amount} audited successfully.")

In [9]:
# Main Transaction Function (uses threads)

def perform_transaction(account, action, amount):
    if action == "deposit":
        account.deposit(amount)
    elif action == "withdraw":
        account.withdraw(amount)

    # Start log and audit in parallel threads

    threading.Thread(target=log_transaction, args=(action, amount), name="Logger").start()
    threading.Thread(target=audit, args=(action, amount), name="Auditor").start()


In [10]:
acc = BankAccount("Dev", 1000)

# Perform multiple transactions concurrently
perform_transaction(acc, "deposit", 500)
perform_transaction(acc, "withdraw", 300)

# Main thread sleeps to allow background threads to finish
time.sleep(2)

[MainThread] Deposited ₹500, New Balance: ₹1500
[MainThread] Withdrew ₹300, New Balance: ₹1200
[LOG] deposit of ₹500 logged.
[LOG] withdraw of ₹300 logged.
[AUDIT] DEPOSIT of ₹500 audited successfully.
[AUDIT] WITHDRAW of ₹300 audited successfully.


What is Multiprocessing?

Multiprocessing is a technique that runs multiple processes simultaneously, each with its own Python interpreter and memory space.


Use it when:
• Tasks are CPU-bound 
(e.g., fraud detection, data encryption, report generation)

• You want to utilize multiple cores for performance

Unlike threading, it bypasses the Global Interpreter Lock (GIL) in CPython.

In [13]:
%%writefile multiprocess_ex.py

import multiprocessing
import time

def calculate_square(n):
    print(f"Calculating square of {n}")
    time.sleep(2)
    print(f"Square of {n} is {n*n}")

if __name__ == "__main__":
    p1 = multiprocessing.Process(target=calculate_square, args=(5,))
    p2 = multiprocessing.Process(target=calculate_square, args=(7,))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("Both calculations completed!")

Writing multiprocess_ex.py


In [14]:
run multiprocess_ex.py

Calculating square of 5
Calculating square of 7
Square of 7 is 49
Square of 5 is 25
Both calculations completed!


True parallelism: runs on separate CPU cores

Bank Application Use Case for Multiprocessing
Imagine a bank needs to:
1.	Process a batch of transactions (deposits/withdrawals)
2.	Perform fraud checks or report generation in parallel

In [15]:
from multiprocessing import Process, Queue, current_process
import time

class BankAccount:
    def __init__(self, name, balance=0):
        self.name = name
        self.balance = balance

    def apply_transaction(self, tx_type, amount):
        if tx_type == 'deposit':
            self.balance += amount
            return f"[{current_process().name}] Deposited ₹{amount}, Balance: ₹{self.balance}"
        elif tx_type == 'withdraw':
            if amount > self.balance:
                return f"[{current_process().name}] Insufficient funds."
            self.balance -= amount
            return f"[{current_process().name}] Withdrew ₹{amount}, Balance: ₹{self.balance}"

In [16]:
#Define Worker Function

def process_transaction(account, tx_type, amount, queue):
    time.sleep(1)  # Simulate processing time
    result = account.apply_transaction(tx_type, amount)
    queue.put(result)  # Return result via queue

In [ ]:
#Main Controller
if __name__ == "__main__":
    from multiprocessing import Queue

    # Create account and a queue to collect results
    acc = BankAccount("Dev", 1000)
    queue = Queue()

    # Create multiple processes for different transactions
    p1 = Process(target=process_transaction, args=(acc, 'deposit', 500, queue), name="DepositProcess")
    p2 = Process(target=process_transaction, args=(acc, 'withdraw', 300, queue), name="WithdrawProcess")

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    # Collect results
    while not queue.empty():
        print(queue.get())

In [18]:
%%writefile multiprocess_bank.py

from multiprocessing import Process, Queue, current_process
import time

class BankAccount:
    def __init__(self, name, balance=0):
        self.name = name
        self.balance = balance

    def apply_transaction(self, tx_type, amount):
        if tx_type == 'deposit':
            self.balance += amount
            return f"[{current_process().name}] Deposited ₹{amount}, Balance: ₹{self.balance}"
        elif tx_type == 'withdraw':
            if amount > self.balance:
                return f"[{current_process().name}] Insufficient funds."
            self.balance -= amount
            return f"[{current_process().name}] Withdrew ₹{amount}, Balance: ₹{self.balance}"


#Define Worker Function

def process_transaction(account, tx_type, amount, queue):
    time.sleep(1)  # Simulate processing time
    result = account.apply_transaction(tx_type, amount)
    queue.put(result)  # Return result via queue


#Main Controller
if __name__ == "__main__":
    from multiprocessing import Queue

    # Create account and a queue to collect results
    acc = BankAccount("Dev", 1000)
    queue = Queue()

    # Create multiple processes for different transactions
    p1 = Process(target=process_transaction, args=(acc, 'deposit', 500, queue), name="DepositProcess")
    p2 = Process(target=process_transaction, args=(acc, 'withdraw', 300, queue), name="WithdrawProcess")

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    # Collect results
    while not queue.empty():
        print(queue.get())

Writing multiprocess_bank.py


In [19]:
run multiprocess_bank.py

[DepositProcess] Deposited ₹500, Balance: ₹1500
[WithdrawProcess] Withdrew ₹300, Balance: ₹700


Important Note:
In multiprocessing, the BankAccount object is copied, not shared. 
So each process gets its own version of the object. 
To share state, you must use 
multiprocessing.Value, 
multiprocessing.Manager, or shared memory.

What is Async I/O?

Async IO lets your program do more while waiting 
— ideal for I/O-bound operations like:
•	API calls
•	Database queries
•	File operations
•	External services (SMS, emails, fraud detection)

Best for:
•	Making many API calls
•	Reading files/networking without blocking
•	Chatbots, crawlers, servers (e.g., FastAPI)

Event Loop in asyncio

•	asyncio provides the event loop that runs async tasks
•	It registers non-blocking I/O operations (like waiting for HTTP responses) and resumes them when ready

In [ ]:
%%writefile asyncio_ex.py

import asyncio

async def greet(name):
    print(f"Hello, {name}")
    await asyncio.sleep(1)  # Non-blocking wait
    print(f"Goodbye, {name}")

async def main():
    await asyncio.gather(
        greet("Surendra"),
        greet("Dev")
    )
asyncio.run(main())

# Run this program from Terminal
# As asyncio.gather() runs both greet() calls concurrently!

Overwriting asyncio_ex.py


Real Example: aiohttp – Async HTTP Requests
Install:
pip install aiohttp

Fetch Multiple URLs in Parallel

In [38]:
%%writefile asyncio_ex2.py

import asyncio
import aiohttp

urls = [
    "https://httpbin.org/delay/2",
    "https://httpbin.org/delay/3",
    "https://httpbin.org/delay/1"
]

async def fetch(session, url):
    async with session.get(url) as response:
        print(f"Fetched {url} with status {response.status}")
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        await asyncio.gather(*tasks)

asyncio.run(main())

Writing asyncio_ex2.py


% python asyncio_ex2.py # Run Program on Terminal
Fetched https://httpbin.org/delay/1 with status 200
Fetched https://httpbin.org/delay/2 with status 200
Fetched https://httpbin.org/delay/3 with status 200
All requests run concurrently, but with a single thread!


Async Bank Application Use Case

Scenario:
•	A user performs a transaction
•	The system must:

1.	Update account balance
2.	Send an SMS notification
3.	Call an external fraud-check API
4.	Send a confirmation email

Let’s do these asynchronously using asyncio and aiohttp.

In [ ]:
#Step 1: Bank Account Logic

class BankAccount:
    def __init__(self, name, balance=0):
        self.name = name
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited ₹{amount}. New balance: ₹{self.balance}")

In [ ]:
#Step 2: Async Notification & Fraud Check

import asyncio
import aiohttp

async def send_sms(name, amount):
    await asyncio.sleep(1)  # simulate delay
    print(f"[SMS] Sent SMS to {name} for ₹{amount} deposit.")

async def send_email(name, amount):
    await asyncio.sleep(2)  # simulate delay
    print(f"[Email] Sent email to {name} confirming ₹{amount} deposit.")

async def fraud_check(name, amount):
    async with aiohttp.ClientSession() as session:
        async with session.get('https://httpbin.org/get') as response:
            data = await response.json()
            print(f"[FraudCheck] Verified transaction for ₹{amount} via {data['url']}")


In [ ]:
#Step 3: Combine with Event Loop

async def perform_transaction(account: BankAccount, amount: float):
    account.deposit(amount)

    # Fire off all async tasks
    await asyncio.gather(
        send_sms(account.name, amount),
        send_email(account.name, amount),
        fraud_check(account.name, amount)
    )

In [ ]:
#Step 4: Run the Event Loop
if __name__ == "__main__":
    acc = BankAccount("Dev", 1000)
    asyncio.run(perform_transaction(acc, 500))

In [39]:
%%writefile asyncio_bankapp.py

#Step 1: Bank Account Logic

class BankAccount:
    def __init__(self, name, balance=0):
        self.name = name
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited ₹{amount}. New balance: ₹{self.balance}")

#Step 2: Async Notification & Fraud Check

import asyncio
import aiohttp

async def send_sms(name, amount):
    await asyncio.sleep(1)  # simulate delay
    print(f"[SMS] Sent SMS to {name} for ₹{amount} deposit.")

async def send_email(name, amount):
    await asyncio.sleep(2)  # simulate delay
    print(f"[Email] Sent email to {name} confirming ₹{amount} deposit.")

async def fraud_check(name, amount):
    async with aiohttp.ClientSession() as session:
        async with session.get('https://httpbin.org/get') as response:
            data = await response.json()
            print(f"[FraudCheck] Verified transaction for ₹{amount} via {data['url']}")



#Step 3: Combine with Event Loop

async def perform_transaction(account: BankAccount, amount: float):
    account.deposit(amount)

    # Fire off all async tasks
    await asyncio.gather(
        send_sms(account.name, amount),
        send_email(account.name, amount),
        fraud_check(account.name, amount)
    )

# Step 4: Run the Event Loop
if __name__ == "__main__":
    acc = BankAccount("Dev", 1000)
    asyncio.run(perform_transaction(acc, 500))

Writing asyncio_bankapp.py



Why Use Profiling?

Before optimizing code, you must know:

•	Which functions are slow?
•	Which lines consume the most CPU?
•	Which lines use too much memory?=

Profiling tools help you measure, not guess.

1. cProfile – Function-Level CPU Profiler

In [40]:
%%writefile my_script.py

import time

def slow_function():
    time.sleep(1)
    return sum(range(1000))

def fast_function():
    return sum(range(10))

def main():
    for _ in range(5):
        slow_function()
        fast_function()

if __name__ == "__main__":
    main()

Writing my_script.py


$python -m cProfile -s cumtime my_script.py

Use it to: Analyze how much time each function takes in total and how many times it is called.

2. line_profiler – Line-by-Line Execution Timing

pip install line_profiler

Decorate the function you want to profile

In [46]:
%%writefile line_test.py

import time

@profile
def sample():
    total = 0
    for i in range(1000):
        total += i
    time.sleep(1)
    return total

if __name__ == "__main__":
    sample()


#Run:
# kernprof -l -v line_test.py

Writing line_test.py


% kernprof -l -v line_test.py

Wrote profile results to line_test.py.lprof
Timer unit: 1e-06 s

Total time: 1.00393 s
File: line_test.py
Function: sample at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           @profile
     5                                           def sample():
     6         1          1.0      1.0      0.0      total = 0
     7      1001        424.0      0.4      0.0      for i in range(1000):
     8      1000        415.0      0.4      0.0          total += i
     9         1    1003086.0    1e+06     99.9      time.sleep(1)

memory_profiler – Line-by-Line Memory Usage

pip install memory_profiler

Decorate with @profile:

In [47]:
%%writefile memory_profiler_ex.py

from memory_profiler import profile

@profile
def memory_hog():
    a = [0] * 1000000  # 1 million zeros
    b = [1] * 1000000
    del b
    return a

memory_hog()

Writing memory_profiler_ex.py


In [48]:
pip install memory_profiler


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#Run
python -m memory_profiler memory_profiler_ex.py

python -m memory_profiler memory_profiler_ex.py
Filename: memory_profiler_ex.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     4     51.2 MiB     51.2 MiB           1   @profile
     5                                         def memory_hog():
     6     58.9 MiB      7.6 MiB           1       a = [0] * 1000000  # 1 million zeros
     7     66.5 MiB      7.6 MiB           1       b = [1] * 1000000
     8     66.5 MiB      0.0 MiB           1       del b
     9     66.5 MiB      0.0 MiB           1       return a